# 3. Python fundamentals II





Grouping, organizing, and reusing code!

* Functions
* Objects
* Modules


## Functions

A function is a chunk of code that takes some inputs, does some processing on them, then returns some outputs. The code within the function runs only when it is called.

Functions can be reused without having to write all the code out again, making your software more **reproducable** and **consistent**. 

Grouping code into functions also keeps your software **organised**.



### Anatomy of a python function

In Python, all functions have a **name**, zero or more **input arguments**, and an **output**. 

We're also going to give all our functions a descriptive **docstring** for this course: I recommend you do the same in your code!

Here's a function with these four basic components:


In [ ]:
def acre_feet_to_m3(volume_acre_feet):
    """Converts volume in US acre-feet to SI m³."""
    volume_m3 = volume_acre_feet * 1_233.482
    return volume_m3

Running this function works just like we've already seen with builtin python functions like `print` and `str`:

In [ ]:
lake_tahoe_volume_acre_feet = 120_000_000
acre_feet_to_m3(lake_tahoe_volume_acre_feet)

Any variables created inside the function are only available inside the function. Try to use `volume_m3` now, you will see an error. 

This scoping of variables inside functions is one of the benefits of functions that keep your workspace clean of variables.


```python
print(volume_m3)
```

```text
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[3], line 1
----> 1 volume_m3

NameError: name 'volume_m3' is not defined
```

### Default arguments

Input arguments can have **default** values. Normally, running a function without specifying all of its arguments results in an error.

But when default values are given for arguments using an `=` sign in the function definition, Python will use that default for any missing arguments.



In [ ]:
def say_hello(user=None):
    if user is None:
        print("Hi!")
    else:
        print(f"Hi {user}!")

In [ ]:
say_hello("Andrew")

In [ ]:
say_hello()

It's nice to give your arguments defaults when you know what the value will be most of the time, but still allow the possibility of using a different input.

### Named arguments

A function will receive the arguments in the order you use them when running the function:

In [ ]:
def bounds_area(left, bottom, right, top, force_positive=True):
    """Area of rectangular bounding box."""
    height = top - bottom
    width = right - left
    area = height * width
    if force_positive:
        area = abs(area)
    return area


bounds_area(4137299, 606008, 4137399, 606009)

Using argument names lets you control the order of the arguments. Plus, will make the code much easier to understand when you're trying to understand it later!

In [ ]:
bounds_area(left=4137299, right=4137399, bottom=606008, top=606009)

In science and geospatial coding, we frequently work between different lat/lon x/y row/col ordering customs, as well as just very complicated algorithms with a large level of parameters.

By writing out argument names in full wherever they're not completely obvious, we help others and our future selves to read our code, and make our code more resilient to bugs.




### Outputs

Technically, Python can only return a single variable as output. But if we want **zero output** we can just return `None` (this is also what happens when we have a function without a `return` statement):


In [ ]:
def check_number_is_non_negative(num):
    """Throw an error for non-negative numbers."""
    assert num >= 0

page_number = 7
result = check_number_is_non_negative(page_number)
print(result)

To stuff **multiple outputs** in one variable, we can use a tuple

In [ ]:
def extract_lat_lon(latlon):
    """Extract lat,lon from a string like '37.364,-122.010'"""
    parts = latlon.split(",")
    lat = float(parts[0])
    lon = float(parts[1])
    return (lat, lon)

result_lat, result_lon = extract_lat_lon("37.364,-122.010")

print(result_lat)

### Type annotations

Modern versions of Python (like the one we're using in our conda environment!) let you document and restrict the types of your input arguments and output using a special syntax.

For the `extract_lat_lon` function above that would look like this:


```python
def extract_lat_lon(latlon: str) -> tuple(float, float):
    ...
```

These **type annotations** are slowly being adopted by many new software projects. But their use isn't widespread in scientific computing.

We chose not to use type annotations for this course as many of the core scientific Python packages we'll be using don't support them (yet!). But you may see them when viewing Python code in the future.



### When to use functions


* When code needs to be reproducable
* WHen code is "done"
    * When you've finished some helper function, scoop all that code into a function and put it at the top of your notebook or in a utils file.
* To avoid repetition
    * Though copy-paste can be better than jamming multiple purposes in one function

### Function decorators



In [ ]:
import functools
import requests

@functools.cache
def load_webpage(url):
    requests.get(url)

In [ ]:
%%time
load_webpage("https://example.com/")

In [ ]:
%%time
load_webpage("https://example.com/about.html")

In [ ]:
%%time
load_webpage("https://example.com/")














1. Functions   
   1. Anatomy of a function  
      1. Arguments (positional, default, keyword, variable)  
      2. Returning outputs  
      3. Variable scope  
      4. Documenting functionality with docstrings  
      5. Type annotations  
   2. Modularization: when to use a function  
   4. Function decorators  
   5. The functools library  
      1. Freezing arguments with functools.partial  
      2. Storing outputs with functools.cache  
3. Object oriented programing  
   1. Objects and classes  
      1. Class syntax  
      2. Instances vs classes  
      3. Methods  
         1. Init  
         2. \_\_str\_\_  
         3. Classmethod and staticmethod  
      4. Property methods  
      5. When to use classes  
      6. OOP for data science  
   2. Dataclasses  
      1. Constructing dataclasses  
      2. Dataclasses vs dictionaries  
   3. Inheritance  
      1. Overriding functionality  
      2. Calling super()  
      3. Extending dataclasses
   3. Debugging nested errors with tracebacks  

2. Modules  
   1. Importing code from other modules  
   2. Module layout  
   3. Auto-reloading imported modules in jupyter  
   4. Packages  
